# Simple simulators

In [ ]:
from itertools import chain
import numpy as np
import random

def generate_cb(i, valid = True):
    import json
    o = {
        "_label_cost": - (i % 2),
        "_label_probability": 0.5,
        "_label_Action": i % 2 + 1,
        "_labelIndex": i % 2,
        "a": [(i % 2) + 1, ((i + 1) % 2) + 1],
        "c": {'shared': {'f':'1'}, '_multi': [{'a': {'f': '1'}}, {'a': {'f': '2'}}]},
        "p": [0.5, 0.5] 
    }
    return f'{json.dumps(o, separators=(",", ":"))}{"" if valid else "!"}\n'


def generate_cb_rnd(i, biases):
    import json
    nusers = biases.shape[0]
    nactions = biases.shape[1]

    user = random.randint(0, nusers - 1)
    chosen = random.randint(0, nactions - 1)

    reward = int(random.random() < biases[user][chosen])
    o = {
        "_label_cost": -reward,
        "_label_probability": 1 / nactions,
        "_label_Action": chosen + 1,
        "_labelIndex": chosen,
        "a": [chosen + 1] + [a + 1 for a in set(range(nactions)) - set([chosen])],
        "c": {'shared': {'f': str(chosen)}, '_multi': [{'a': {'f': str(i)}} for i in range(nactions)]},
        "p": [1.0 / nactions]  * nactions
    }
    return f'{json.dumps(o, separators=(",", ":"))}\n'

def generate_ccb(i):
    import json
    o = {
        'c': {'context': {'f':'1'}, '_multi': [{'a': {'f': '1'}}, {'a': {'f': '2'}}, {'a': {'f': '3'}}], '_slots': [{'s': {'f': '1'}}, {'s': {'f': '2'}}]},
        '_outcomes':[{'_label_cost': -i % 2, '_a': [(i + 2) % 3, i, (i + 1) % 3], '_p': [0.5, 0.25, 0.25]}, {'_label_cost': - (i + 1) % 2, '_a': [i, (i + 1) % 3], '_p': [0.5, 0.5]}],
    }
    return f'{json.dumps(o, separators=(",", ":"))}\n'

def generate_file(generator, lines, path):
    with open(path, 'w') as f:
        f.writelines(map(lambda i: generator(i), range(lines)))

def generate_file_with_invalid(lines, path):
    with open(path, 'w') as f:
        for i in range(lines):
            f.write(generate_cb(i))
        f.write(generate_cb(0, False))
        for i in range(lines):
            f.write(generate_cb(i))          


In [ ]:
generate_file(generate_cb, 10, 'vw_executor/tests/data/cb_0.json')
generate_file(generate_cb, 11, 'vw_executor/tests/data/cb_1.json')
generate_file_with_invalid(10, 'vw_executor/tests/data/cb_invalid.json')

biases = np.random.rand(4,4)
print(biases)
generate_file(lambda i: generate_cb_rnd(i, biases), 10000, 'vw_executor/tests/data/cb_10000_0.json')
generate_file(lambda i: generate_cb_rnd(i, biases), 10000, 'vw_executor/tests/data/cb_10000_1.json')

In [ ]:
generate_file(generate_ccb, 10, 'vw_executor/tests/data/ccb_0.json')
generate_file(generate_ccb, 11, 'vw_executor/tests/data/ccb_1.json')

# More realistic simulator

In [7]:
import random
import pandas as pd
import numpy as np
import json
import scipy

class Simulation(pd.DataFrame):
    def __init__(self, n=10000, swap_after=5000, variance = 0, bad_features = 0):
        examples = []
        offset = 0
        for i in range(1, n):
            if i % swap_after == 0:
                offset = (offset + 1) % 2

            person = i % 2
            chosen = i % 4  // 2
            if (chosen + person + offset) % 2 == 0:
                reward =  0.7 + ((chosen + offset) % 2) * 0.1
            else:
                reward = 0.3 - ((chosen + offset + 1) % 2) * 0.1

            reward = reward + scipy.random.normal(0, variance)

            examples.append({
                "reward": reward,
                "shared_good": person,
                "a1_good": '0',
                "a2_good": '1',
                "chosen": chosen, 
                "prob": 0.5
            })
            for i in range(bad_features):
                examples[-1][f'a1_bad_{i}'] = random.random()
                examples[-1][f'a2_bad_{i}'] = random.random()
        super().__init__(examples)
    
    def get(self):
        return self.df
    
    def to_dsjson(self):
        for _, row in self.iterrows():
            o = {
                "_label_cost": -row['reward'],
                "_label_probability": row['prob'],
                "_label_Action": int(row['chosen'] + 1),
                "_labelIndex": int(row['chosen']),
                "a": [row['chosen'] + 1, (row['chosen'] + 1) % 2 + 1],
                "c": {'shared': {'f': str(row['shared_good'])},
                    '_multi': [{'a': {'f': str(row['a1_good']), 'constant': 1}},
                        {'a': {'f': str(row['a2_good'])}}]},
                "p": [row['prob'], 1 - row['prob']] 
            }
            i = 0
            while True:
                if f'a1_bad_{i}' in row:
                    if 'b' not in o['c']['_multi'][0]: 
                        o['c']['_multi'][0]['b'] = {}
                        o['c']['_multi'][1]['b'] = {}   
                    o['c']['_multi'][0]['b'][f'a1_bad_{i}'] = row[f'a1_bad_{i}']
                    o['c']['_multi'][1]['b'][f'a2_bad_{i}'] = row[f'a2_bad_{i}']
                    i += 1
                else:
                    break
            yield json.dumps(o, separators=(",", ":"))

    def to_dsjson_file(self, fname):
        with open(fname, 'w') as f:
            for ex in self.to_dsjson():
                f.write(f'{ex}\n')             

In [10]:
simulation = Simulation(1000, 500, 0.1, 2)
simulation.to_dsjson_file('file.json')